In [9]:
from scipy import optimize
from scipy import interpolate as interp
import numpy as np
from skimage import measure


In [32]:
theta = np.load('../../output_data/material_Ti64/power_200_velocity_0.5/absorp_0.4/meltpool_timestep_3.npy')

In [36]:
melt_T = 1700
verbose = False
b = 200e-6
bounds = {'x': [-1000e-6, 1000e-6], 'y': [-300e-6, 300e-6], 'z': [-400e-6, 0]}
dimstep = 5e-6
xs = np.arange(-b + bounds['x'][0], bounds['x'][1]+ b, step = dimstep)
ys = np.arange(bounds['y'][0] - b, bounds['y'][1] + b, step = dimstep)
zs = np.arange(bounds['z'][0], bounds['z'][1] + dimstep, step = dimstep)


In [37]:
def calculate_dimensions(theta, mesh, melt_T, dimstep, verbose = False):
    """
    Calculate the dimensions of a melt pool based on temperature data.

    Args:
        theta (ndarray): 3D numpy array representing the temperature distribution.
        mesh (tuple): Tuple containing the x, y, and z coordinates of the mesh. 
        Each entry is a numpy array in ascending order containing the unique x, y, or z coordinates of the mesh respectively.
        melt_T (float): Melting temperature of the material.
        dimstep (float): Step size of the mesh.

    Returns:
        dict: Dictionary containing the calculated dimensions of the melt pool.
            - 'length': Length of the melt pool.
            - 'width': Width of the melt pool.
            - 'depth': Depth of the melt pool.
    """
    
    xs, ys, zs = mesh
    # center of the melt pool is where it's the hottest
    dimensions = {}
    if not np.array(theta[:,:,-1] > melt_T).any():
        print(f"Energy Density too low to melt material, melting temperature: {melt_T} K, max temperature: {np.max(theta[:,:,-1])} K")
        length = 0
        width = 0
        depth = 0
        dimensions['length'] = length
        dimensions['width'] = width 
        dimensions['depth'] = depth
        return dimensions
    else:

        # Calculate length and width the easy way
        # Create object to calculate the dimensions of the binarized melt pool
        prop = measure.regionprops(np.array(theta[:,:,-1]>melt_T, dtype = 'int'))
        length = prop[0].major_axis_length*dimstep

        if verbose:
            print("Length: {:.04} ± {:.04}".format(length*1e6, dimstep*1e6))
        width = prop[0].minor_axis_length*dimstep
        if verbose:
            print("Width: {:.04} ± {:.04}".format(width*1e6, dimstep*1e6))

        depths = []
        for j in range(len(ys)):
            for i in range(len(xs)):     
                # Calculate depth the hard way
                if theta[i, j, -1] > melt_T:
                    g = interp.CubicSpline(zs, theta[i, j, :] - melt_T)
                    root = optimize.brentq(g, zs[0],zs[-1])
                    depths.append(root)

        depth = np.min(depths)
        dimensions['length'] = length
        dimensions['width'] = width
        dimensions['depth'] = -depth
        if verbose:
            print("Calculated dimensions. Length: {:.04}µm, Width: {:.04}µm, Depth: {:.04}µm".format(length*1e6, width*1e6, -depth*1e6))
        return dimensions
    


In [35]:
mesh = (xs, ys, zs)
dimensions = calculate_dimensions(theta,
                                  mesh,
                                  melt_T,
                                  dimstep = 5e-6,
                                  verbose = True)
print(dimensions)

Length: 539.8 ± 5.0
Width: 183.6 ± 5.0
Calculated dimensions. Length: 539.8µm, Width: 183.6µm, Depth: 79.74µm
{'length': 0.0005398443421684037, 'width': 0.00018355961652154616, 'depth': 7.974130916267422e-05}
